In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from delta.tables import DeltaTable
import os

* clone the accounts table
* enable deletion vectors on it
* merge the increment into it

In [ ]:
spark = (
    SparkSession
    .builder
    .appName('delta-III')
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

In [ ]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

questions = os.path.join(project_path, 'data/questions')
users_base_path = os.path.join(project_path, 'data/users_base')
users_increment_path = os.path.join(project_path, 'data/users_increment')
accounts_output_path = os.path.join(project_path, 'output/delta/accounts')

In [ ]:
spark.sql('drop table if exists accounts')

In [ ]:
spark.sql(f"""
    CREATE TABLE accounts
    USING DELTA
    LOCATION '{accounts_output_path}'
""")

In [ ]:
spark.sql('ALTER TABLE accounts SET TBLPROPERTIES (delta.enableDeletionVectors = true)')

In [ ]:
spark.sql('desc detail accounts').select('properties', 'minReaderVersion', 'minWriterVersion').show(n=100, truncate=100)

In [ ]:
increment = spark.read.parquet(users_increment_path)

In [ ]:
(
    DeltaTable.forName(spark, 'accounts')
    .alias('accounts')
    .merge(
        increment.alias('increment'),
        'accounts.user_id == increment.user_id'
    )
    .whenMatchedUpdateAll()
    .whenNotMatchedInsertAll()
    .execute()
)